<a href="https://colab.research.google.com/github/n-beckage/CS254_Final_Project/blob/main/FeatureEngineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import datetime
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from matplotlib import pyplot as plt
import re
import collections

from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import os

# TODO: Fill in the Google Drive path where you uploaded the assignment, it should be under CS254/CS254-Assignments/netid-A2
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'CS254/CS 254 Final Project/' # change this directory to yours
GOOGLE_DRIVE_PATH = os.path.join('drive', 'MyDrive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

['nytcrosswords.csv', 'ML Final Project Proposal Presentation.gslides', 'Project_Proposal_Report_Group_2.gdoc', 'Features to engineer.gdoc', 'CS Fair Slides.gslides', 'NestedCV_AllParams.ipynb', 'OLDFeatureEngineering.ipynb', 'OLDcrosswords_engineered.csv', 'crosswords_engineered.csv', 'RandomizedGridCV_rbf_poly.ipynb', 'RandomizedGridCV_AllParams.ipynb', 'Copy of CS254 - Project Final Report Format.gdoc', 'FeatureEngineering.ipynb', 'NestedCV_rbf_poly.ipynb']


In [3]:
import io
import string
import csv


# Defining classes to be used
class Clue:
    def __init__(self, question, answer, date):
        self.question = question
        self.answer = answer
        self.date = date
    def __str__(self):
        return str("Question: " + self.question + " Answer: " + self.answer)

class Puzzle:
    questions = []

    #Feature field here
    def __init__(self, date, questions):
        self.questions = questions
        self.date = date
        self.day_of_week = self.get_day()
        self.average_question_length = self.get_average_question_length()
        self.average_answer_length = self.get_average_answer_length()
        self.num_questions = len(questions)

        self.longest_question = self.get_longest_question()
        self.shortest_question = self.get_shortest_question()
        self.longest_answer = self.get_longest_answer()
        self.shortest_answer = self.get_shortest_answer()
        self.question_mark_count = self.get_question_mark_count()
        self.exclamation_count = self.get_exclamation_count()
        self.count_ending_with_question = self.get_questions_ending_with_mark()

        self.a_freq = self.get_a_freq()
        self.b_freq = self.get_b_freq()
        self.c_freq = self.get_c_freq()
        self.d_freq = self.get_d_freq()
        self.e_freq = self.get_e_freq()
        self.f_freq = self.get_f_freq()
        self.g_freq = self.get_g_freq()
        self.h_freq = self.get_h_freq()
        self.i_freq = self.get_i_freq()
        self.j_freq = self.get_j_freq()
        self.k_freq = self.get_k_freq()
        self.l_freq = self.get_l_freq()
        self.m_freq = self.get_m_freq()
        self.n_freq = self.get_n_freq()
        self.o_freq = self.get_o_freq()
        self.p_freq = self.get_p_freq()
        self.q_freq = self.get_q_freq()
        self.r_freq = self.get_r_freq()
        self.s_freq = self.get_s_freq()
        self.t_freq = self.get_t_freq()
        self.u_freq = self.get_u_freq()
        self.v_freq = self.get_v_freq()
        self.w_freq = self.get_w_freq()
        self.x_freq = self.get_x_freq()
        self.y_freq = self.get_y_freq()
        self.z_freq = self.get_z_freq()

        self.long_answers = self.get_long_answers()
        self.short_answers = self.get_short_answers()
        self.mid_answers = self.get_mid_answers()
        self.sp_questions = self.get_sp_questions()
        self.fr_questions = self.get_fr_questions()
        self.abbr_questions = self.get_abbr_questions()
        self.avg_vow_answers = self.get_avg_vow_answers()
        self.duplicate_questions = self.get_duplicate_questions()
        self.duplicate_answers = self.get_duplicate_answers()
        self.pronouns_ques = self.get_pronouns_ques()
        self.cap_let_questions = self.get_cap_let_questions()

        # Feature field = self.feature_function()



    def __str__(self):
        return "Date: " + self.date + " Questions: " + str(len(self.questions)) 

    ### Define a feature_function here and how to calculate it

    # get day of the week from dates (TARGET)
    def get_day(self):
        month, day, year = (int(x) for x in self.date.split('/'))   
        my_date = datetime.date(year, month, day)
        return my_date.strftime("%A")  #'Wednesday'

    def get_average_question_length(self):
        total = 0.0
        counted = 0.0
        for clue in self.questions:
            total += len(clue.question)
            counted += 1
        return total/counted
    
    def get_average_answer_length(self):
        total = 0.0
        counted = 0.0
        for answer in self.questions:
            total += len(answer.answer)
            counted += 1
        return total/counted

    def get_longest_question(self):
      max_length = 0
      for clue in self.questions:
        if len(clue.question) > max_length:
          max_length = len(clue.question)
        else:
          max_length = max_length
      return max_length

    def get_shortest_question(self):
      min_length = 100000
      for clue in self.questions:
        if len(clue.question) < min_length:
          min_length = len(clue.question)
        else:
          min_length = min_length
      return min_length

    
    def get_longest_answer(self):
      max_length = 0
      for answer in self.questions:
        if len(answer.answer) > max_length:
          max_length = len(answer.answer)
        else:
          max_length = max_length
      return max_length

    def get_shortest_answer(self):
      min_length = 100000
      for answer in self.questions:
        if len(answer.answer) < min_length:
          min_length = len(answer.answer)
        else:
          min_length = min_length
      return min_length
    
    def get_letters_in_puzzle(self):
      total_letters = 0
      for answer in self.questions:
        answer_letters = len(answer.answer)-answer.answer.count(' ')
        total_letters += answer_letters 
      return total_letters

    def get_question_mark_count(self):
      count = 0
      for clue in self.questions:
        count += clue.question.count("?")
      return count

    def get_exclamation_count(self):
      count = 0
      for clue in self.questions:
        count += clue.question.count("!")
      return count

    def get_questions_ending_with_mark(self):
      count = 0
      for clue in self.questions:
        if clue.question.endswith('?'):
          count+=1
        else:
          count=count
      return count

    # functions to get letter frequencies for puzzle answers
    def get_a_freq(self):
      count = 0
      total_len = 0
      for answer in self.questions:
          count += answer.answer.count("A")
          total_len += len(answer.answer)
      a_freq = 100 * count / total_len
      return a_freq
      
    def get_b_freq(self):
      count = 0
      total_len = 0
      for answer in self.questions:
          count += answer.answer.count("B")
          total_len += len(answer.answer)
      b_freq = 100 * count / total_len
      return b_freq

    def get_c_freq(self):
      count = 0
      total_len = 0
      for answer in self.questions:
          count += answer.answer.count("C")
          total_len += len(answer.answer)
      c_freq = 100 * count / total_len
      return c_freq

    def get_d_freq(self):
      count = 0
      total_len = 0
      for answer in self.questions:
          count += answer.answer.count("D")
          total_len += len(answer.answer)
      d_freq = 100 * count / total_len
      return d_freq

    def get_e_freq(self):
      count = 0
      total_len = 0
      for answer in self.questions:
          count += answer.answer.count("E")
          total_len += len(answer.answer)
      e_freq = 100 * count / total_len
      return e_freq

    def get_f_freq(self):
      count = 0
      total_len = 0
      for answer in self.questions:
          count += answer.answer.count("F")
          total_len += len(answer.answer)
      f_freq = 100 * count / total_len
      return f_freq

    def get_g_freq(self):
      count = 0
      total_len = 0
      for answer in self.questions:
          count += answer.answer.count("G")
          total_len += len(answer.answer)
      g_freq = 100 * count / total_len
      return g_freq

    def get_h_freq(self):
      count = 0
      total_len = 0
      for answer in self.questions:
          count += answer.answer.count("H")
          total_len += len(answer.answer)
      h_freq = 100 * count / total_len
      return h_freq

    def get_i_freq(self):
      count = 0
      total_len = 0
      for answer in self.questions:
          count += answer.answer.count("I")
          total_len += len(answer.answer)
      i_freq = 100 * count / total_len
      return i_freq

    def get_j_freq(self):
      count = 0
      total_len = 0
      for answer in self.questions:
          count += answer.answer.count("J")
          total_len += len(answer.answer)
      j_freq = 100 * count / total_len
      return j_freq

    def get_k_freq(self):
      count = 0
      total_len = 0
      for answer in self.questions:
          count += answer.answer.count("K")
          total_len += len(answer.answer)
      k_freq = 100 * count / total_len
      return k_freq

    def get_l_freq(self):
      count = 0
      total_len = 0
      for answer in self.questions:
          count += answer.answer.count("L")
          total_len += len(answer.answer)
      l_freq = 100 * count / total_len
      return l_freq

    def get_m_freq(self):
      count = 0
      total_len = 0
      for answer in self.questions:
          count += answer.answer.count("M")
          total_len += len(answer.answer)
      m_freq = 100 * count / total_len
      return m_freq

    def get_n_freq(self):
      count = 0
      total_len = 0
      for answer in self.questions:
          count += answer.answer.count("N")
          total_len += len(answer.answer)
      n_freq = 100 * count / total_len
      return n_freq

    def get_o_freq(self):
      count = 0
      total_len = 0
      for answer in self.questions:
          count += answer.answer.count("O")
          total_len += len(answer.answer)
      o_freq = 100 * count / total_len
      return o_freq

    def get_p_freq(self):
      count = 0
      total_len = 0
      for answer in self.questions:
          count += answer.answer.count("P")
          total_len += len(answer.answer)
      p_freq = 100 * count / total_len
      return p_freq

    def get_q_freq(self):
      count = 0
      total_len = 0
      for answer in self.questions:
          count += answer.answer.count("Q")
          total_len += len(answer.answer)
      q_freq = 100 * count / total_len
      return q_freq

    def get_r_freq(self):
      count = 0
      total_len = 0
      for answer in self.questions:
          count += answer.answer.count("R")
          total_len += len(answer.answer)
      r_freq = 100 * count / total_len
      return r_freq
  
    def get_s_freq(self):
      count = 0
      total_len = 0
      for answer in self.questions:
          count += answer.answer.count("S")
          total_len += len(answer.answer)
      s_freq = 100 * count / total_len
      return s_freq

    def get_t_freq(self):
      count = 0
      total_len = 0
      for answer in self.questions:
          count += answer.answer.count("T")
          total_len += len(answer.answer)
      t_freq = 100 * count / total_len
      return t_freq

    def get_u_freq(self):
      count = 0
      total_len = 0
      for answer in self.questions:
          count += answer.answer.count("U")
          total_len += len(answer.answer)
      u_freq = 100 * count / total_len
      return u_freq

    def get_v_freq(self):
      count = 0
      total_len = 0
      for answer in self.questions:
          count += answer.answer.count("V")
          total_len += len(answer.answer)
      v_freq = 100 * count / total_len
      return v_freq

    def get_w_freq(self):
      count = 0
      total_len = 0
      for answer in self.questions:
          count += answer.answer.count("W")
          total_len += len(answer.answer)
      w_freq = 100 * count / total_len
      return w_freq
  
    def get_x_freq(self):
      count = 0
      total_len = 0
      for answer in self.questions:
          count += answer.answer.count("X")
          total_len += len(answer.answer)
      x_freq = 100 * count / total_len
      return x_freq

    def get_y_freq(self):
      count = 0
      total_len = 0
      for answer in self.questions:
          count += answer.answer.count("Y")
          total_len += len(answer.answer)
      y_freq = 100 * count / total_len
      return y_freq

    def get_z_freq(self):
      count = 0
      total_len = 0
      for answer in self.questions:
          count += answer.answer.count("Z")
          total_len += len(answer.answer)
      z_freq = 100 * count / total_len
      return z_freq

    # additional features
    def get_long_answers(self):
      count = 0
      for answer in self.questions:
        if len(answer.answer) > 6:
          count += 1
      return count

    def get_short_answers(self):
      count = 0
      for answer in self.questions:
        if len(answer.answer) < 4:
          count += 1
      return count

    def get_mid_answers(self):
      count = 0
      for answer in self.questions:
        if len(answer.answer) >= 4 and len(answer.answer) <= 6:
          count += 1
      return count

    def get_sp_questions(self):
      count = 0
      for clue in self.questions:
        if clue.question.endswith("sp.") == 1 or clue.question.endswith("Sp.") == 1:
          count += 1
      return count

    def get_fr_questions(self):
      count = 0
      for clue in self.questions:
        if clue.question.endswith("fr.") == 1 or clue.question.endswith("Fr.") == 1:
          count += 1
      return count      

    def get_abbr_questions(self):
      count = 0
      for clue in self.questions:
        if clue.question.endswith("abbr.") == 1 or clue.question.endswith("Abbr.") == 1:
          count += 1
      return count

    def get_avg_vow_answers(self):
      count = 0
      for answer in self.questions:
          count += len(re.findall("a|e|i|o|u",answer.answer.lower()))
      avg = count/len(self.questions)
      return avg


    def get_duplicate_questions(self):
      num_dups = 0
      questions_counter = collections.Counter([clue.question for clue in self.questions])
      for question,count in questions_counter.items():
        if count > 1:
          num_dups += 1
      return num_dups

    def get_duplicate_answers(self):
      num_dups = 0
      answers_counter = collections.Counter([answer.answer for answer in self.questions])
      for question,count in answers_counter.items():
        if count > 1:
          num_dups += 1
      return num_dups     

    def get_pronouns_ques(self):
      count = 0 
      for clue in self.questions:
        if len(re.findall(" he | him | she | her | they | them ",clue.question.lower())) >= 1:
          count += 1
      return count

    def get_cap_let_questions(self):
      count = 0
      for clue in self.questions:
        count += len(re.findall(r'[A-Z]', clue.question))
      return count  


#questions_and_answers is the list of all Clue objects to be grouped later
questions_and_answers = []

# Opening the file and reading the questions into Clues
with open(GOOGLE_DRIVE_PATH + "nytcrosswords.csv", newline='', encoding='windows_1258') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        date = row[0]
        answer = row[1]
        clue = row[2]
        new_clue = Clue(clue, answer, date)
        questions_and_answers.append(new_clue)

# Removing the header column
questions_and_answers.remove(questions_and_answers[0])


# This is the final list of puzzles
puzzleList = []
# questionList is the list of questions of the same date about 
# to be added to a puzzle object
cluesToAdd = []

# Placing all of the Clue objects into a list of Puzzle objects
currentDate = questions_and_answers[0].date
for i in range(len(questions_and_answers)):
    if(questions_and_answers[i].date) == currentDate:
        cluesToAdd.append(questions_and_answers[i])
    else:
        new_puzzle = Puzzle(currentDate, cluesToAdd.copy())
        puzzleList.append(new_puzzle)

        cluesToAdd.clear()

        currentDate = questions_and_answers[i].date
        cluesToAdd.append(questions_and_answers[i])


# Writing the Puzzles into a file
fields = ['Date', 'TARGET', 'Average Question Length', 'avg_answer_length','Number of Questions', 'longest_question', 'shortest_question', 'longest_answer', 'shortest_answer', 
          'question_mark_count', 'exclamation_count', 'count_ending_with_question', 'a_freq', 'b_freq', 'c_freq', 'd_freq', 'e_freq', 'f_freq', 'g_freq', 'h_freq',
          'i_freq', 'j_freq', 'k_freq', 'l_freq', 'm_freq', 'n_freq', 'o_freq', 'p_freq', 'q_freq', 'r_freq', 's_freq', 't_freq', 'u_freq', 'v_freq', 'w_freq',
          'x_freq', 'y_freq', 'z_freq', 'long_answers', 'short_answers', 'mid_answers', 'sp_questions', 'fr_questions', 'abbr_questions', 'avg_vow_answers', 'duplicate_questions',
          'duplicate_answers', 'pronouns_ques', 'cap_let_questions']

rows = []
for puzzle in puzzleList:
    #THIS IS WHERE WE ADD FEATURES TO THE FINAL OUTPUT
    rows.append([puzzle.date, puzzle.day_of_week, puzzle.average_question_length, puzzle.average_answer_length, puzzle.num_questions, puzzle.longest_question, puzzle.shortest_question, 
                 puzzle.longest_answer, puzzle.shortest_answer, puzzle.question_mark_count, puzzle.exclamation_count, puzzle.count_ending_with_question, puzzle.a_freq,
                 puzzle.b_freq, puzzle.c_freq, puzzle.d_freq, puzzle.e_freq, puzzle.f_freq, puzzle.g_freq, puzzle.h_freq, puzzle.i_freq, puzzle.j_freq, puzzle.k_freq,
                 puzzle.l_freq, puzzle.m_freq, puzzle.n_freq, puzzle.o_freq, puzzle.p_freq, puzzle.q_freq, puzzle.r_freq, puzzle.s_freq, puzzle.t_freq, puzzle.u_freq,
                 puzzle.v_freq, puzzle.w_freq, puzzle.x_freq, puzzle.y_freq, puzzle.z_freq, puzzle.long_answers, puzzle.short_answers, puzzle.mid_answers, puzzle.sp_questions,
                 puzzle.fr_questions, puzzle.abbr_questions, puzzle.avg_vow_answers, puzzle.duplicate_questions, puzzle.duplicate_answers, puzzle.pronouns_ques, puzzle.cap_let_questions])

with open(GOOGLE_DRIVE_PATH + 'crosswords_engineered.csv', 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(fields)
    csvwriter.writerows(rows)


# Features to Build
- Number of words in the clue
- Does clue contain pronoun?
- Weight letters in the clue by frequencey
  https://en.wikipedia.org/wiki/Letter_frequency
- Weights letter in answer by frequency 

### Other tasks
- Figure out what day of the week each clue is from and filter the dataset to just be Sunday-Wednesday puzzle clues

### Labels
- Name of a person
- Too complicated without NLP (basically our 'other' category
- 



# Loading the Dataset
The dataset is located in our folder

In [4]:
df = pd.read_csv(GOOGLE_DRIVE_PATH+"nytcrosswords.csv", encoding="windows_1258")
df.head() #look at the first few rows

,Date,Word,Clue
0,10/31/2021,PAT,"Action done while saying ""Good dog"""
1,10/31/2021,RASCALS,Mischief-makers
2,10/31/2021,PEN,It might click for a writer
3,10/31/2021,SEP,Fall mo.
4,10/31/2021,ECO,Kind to Mother Nature


# Start of Feature Extraction + Engineering

---

HEY TEAM! Here's what I've done so far to code feature extraction, hope this is a good headstart! I also made an outline for next steps for reliminary results in text boxes below.
Message me with questions and lmk if you want to code more of it with me this weekend! 😀 - Anna Jane

In [5]:
# get given features from dataframe (feature extraction)
clues = np.asarray(df["Clue"], dtype=str)
answers = np.asarray(df["Word"], dtype=str)
# found an error:  when answer to a clue is "NULL" python replaces it with a NaN.
# here I find all instances where the answer was replaced with a NaN and restate 
# the answer to the clue as "NULL".
for i in np.argwhere(answers=="nan"):
    answers[i]="NULL"

dates = np.asarray(df["Date"])

In [ ]:
# get day of the week from dates (getting targets/labels)
def get_days(dates):
    clue_days = []
    for i in range(len(dates)):
        month, day, year = (int(x) for x in dates[i].split('/'))   
        my_date = datetime.date(year, month, day)
        clue_days.append(my_date.strftime("%A"))  #'Wednesday'
    return np.array(clue_days)

# call function
clue_days = get_days(dates) 
# ^ ideally want to condense these to "puzzle_days" since each example we feed our classifier will be one puzzle

In [ ]:
print(clue_days)

In [ ]:
# (feature engineering)

# get clue lengths: number of characters (including punctuation) in clue
clue_lengths = np.empty(len(clues))
for i in range(len(clue_lengths)):
    clue_lengths[i] = len(clues[i])

# get answer lengths: number of characters (including punctuation) in answer
answer_lengths = np.empty(len(answers))
for i in range(len(answer_lengths)):
    answer_lengths[i] = len(answers[i])

## Splitting the data into train/test data
Our simplest case model - simple SVM with 80% train 20% test

In [ ]:
days = get_days(dates)
len(days)

In [ ]:
# putting the data into a nx3 arrray where n is the number of rows (781,573). Every time this cell is run the data is re-ordered
data_arr = np.empty((len(days),3), dtype=object)
for i in range(len(days)):
  data_arr[i][0] = clue_lengths[i]
  data_arr[i][1] = answer_lengths[i]
  data_arr[i][2] = days[i]

In [ ]:
# this is an inplace function to SHUFFLE DATA
np.random.shuffle(data_arr)

In [ ]:
def get_subset(data_array, size):
  return data_array[:size]

# getting the subset of our data (non-random, takes n frist values of data_arr)
subset = get_subset(data_arr, 1000)

# Training an SVM Classifier
After feature extraction/engineering and splitting up data into features for each puzzle, train an SVM on a small set of examples.

- Create an <a href="https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html">SVC</a> object, imported from sklearn's svm module, use linear or poly kernel.
- Call the .fit() function on your model with x_train and y_train as parameters.
- Call the .predict() function on your model with x_test as a parameter and store the results.

In [ ]:
# test_cell

# x_train, x_test, y_train, y_test = train_test_split(subset[:,0:2], subset[:,2], test_size = 0.20, random_state=0)
# supVect.fit(x_train, y_train)
# print(supVect.score(x_test,y_test))
# y_preds = supVect.predict(x_test)
# print(accuracy_score(y_test, y_preds))
# print(classification_report(y_test, y_preds))
len(list(range(1000, 10500, 500)))

In [ ]:
import warnings
warnings.filterwarnings('ignore')
# TAKES ~5 MINUTES TO RUN - BUILDING MODELS
supVect = SVC(kernel = 'rbf') # this is the default kernel
test_errors = []
train_errors = []
test_reports = []
train_reports = []
iters = range(1000, 10250, 250)
for i in iters:
  np.random.shuffle(data_arr)
  subset = get_subset(data_arr, i)
  x_train, x_test, y_train, y_test = train_test_split(subset[:,0:2], subset[:,2], test_size = 0.20, random_state=0)
  supVect.fit(x_train, y_train)
  y_test_preds = supVect.predict(x_test)
  test_score = accuracy_score(y_test, y_test_preds)
  test_error = 1 - test_score
  test_errors.append(test_error)
  y_train_preds = supVect.predict(x_train)
  train_score = accuracy_score(y_train, y_train_preds)
  train_error = 1 - train_score
  train_errors.append(train_error)
  test_reports.append(classification_report(y_test, y_test_preds))
  train_reports.append(classification_report(y_train, y_train_preds))

### Summary Statistics for All Models

In [ ]:
print("Number of models created:",len(iters))
print("Starting at m =",iters[0],"up to m =",iters[len(iters)-1],"at increments of",iters[1]-iters[0])
print("Average test error =",np.mean(test_errors))
print("Average test accuracy =",1-np.mean(test_errors))
print("Average training error =",np.mean(train_errors))
print("Average training accuracy =",1-np.mean(train_errors))

### Best SVM Model Statistics

In [ ]:
best_idx = test_errors.index(min(test_errors))
print("Model with the highest test accuracy had M =",iters[best_idx])
print("Test error was",test_errors[best_idx])
print("Training error was",train_errors[best_idx])
print("\n TEST REPORT:")
print(test_reports[best_idx])
print("\n TRAINING REPORT:")
print(train_reports[best_idx])

### Worst SVM Model Statistics

In [ ]:
worst_idx = test_errors.index(max(test_errors))
print("Model with the lowest test accuracy had M =",iters[worst_idx])
print("Test error was",test_errors[worst_idx])
print("Training error was",train_errors[worst_idx])
print("\n TEST REPORT:")
print(test_reports[worst_idx])
print("\n TRAINING REPORT:")
print(train_reports[worst_idx])

In [ ]:
# Counting the balance of our data
import collections
elements_count = collections.Counter(clue_days)
for key, value in elements_count.items():
   print(f"{key}: {round((value/len(clue_days))*100,1)}%")

In [ ]:
print(list(iters))
print(test_errors)

In [ ]:
plt.rcParams["figure.figsize"] = (10,6)
plt.plot(list(iters),train_errors)
plt.plot(list(iters),test_errors, color = "green")
plt.xlabel("Sample Size (M)")
plt.ylabel("Error")
plt.legend(["Train","Test"])

In [ ]:
supVect.score(x_train,y_train)